## Basic Building Blocks of ML: Tensors

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

/Users/timlee/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [2]:
m1 = [[1.0, 2.], [3., 4.]]; print(m1 , type(m1))

[[1.0, 2.0], [3.0, 4.0]] <class 'list'>


In [3]:
m2 = np.array(m1, dtype=np.float32); print(m2, type(m2))

[[1. 2.]
 [3. 4.]] <class 'numpy.ndarray'>


In [4]:
m3 = tf.constant(m1); print(m3, type(m3))

Tensor("Const:0", shape=(2, 2), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>


### Playing around with TF constants

In [5]:
m4 = tf.constant(55, name='alpha')
print(m4)
m5 = tf.constant([55], name='beta')
print(m5)
m6 = tf.constant([1,2], name='gamma')
print(m6)
m7 = tf.constant([[1],[2]], name='delta')
print(m7)
m8 = tf.constant([[1,2], [3,4]], name='epsilon')
print(m8)
m9 = tf.constant([[[1,2], [3,4]], [[5,6], [7,8]]], name='lambda')
print(m9)

Tensor("alpha:0", shape=(), dtype=int32)
Tensor("beta:0", shape=(1,), dtype=int32)
Tensor("gamma:0", shape=(2,), dtype=int32)
Tensor("delta:0", shape=(2, 1), dtype=int32)
Tensor("epsilon:0", shape=(2, 2), dtype=int32)
Tensor("lambda:0", shape=(2, 2, 2), dtype=int32)


### Data Types
|Data type | Python Type | Description |
| ----- | ----- | ------|
|DT_FLOAT|tf.float32|32 bits floating point.|
|DT_DOUBLE|tf.float64|64 bits floating point.|
|DT_INT8|tf.int8|8 bits signed integer.|
|DT_INT16|tf.int16|16 bits signed integer.|
|DT_INT32|tf.int32|32 bits signed integer.|
|DT_INT64|tf.int64|64 bits signed integer.|
|DT_UINT8|tf.uint8|8 bits unsigned integer.|
|DT_STRING|tf.string|Variable length byte arrays. Each element of a Tensor is a byte array.|
|DT_BOOL|tf.bool|Boolean.|
|DT_COMPLEX64|tf.complex64|Complex number made of two 32 bits floating points: real and imaginary parts.|
|DT_COMPLEX128|tf.complex128|Complex number made of two 64 bits floating points: real and imaginary parts.|
|DT_QINT8|tf.qint8|8 bits signed integer used in quantized Ops.|
|DT_QINT32|tf.qint32|32 bits signed integer used in quantized Ops.|
|DT_QUINT8|tf.quint8|8 bits unsigned integer used in quantized Ops.|


# Graphs & Sessions

## Graphs
>TensorFlow uses a **dataflow graph** to represent your computation in terms of the dependencies between individual operations. This leads to a low-level programming model in which you first define the dataflow graph, then create a TensorFlow session to run parts of the graph across a set of local and remote devices. The graph has two main components:

> **Graph structure.** The nodes and edges of the graph, indicating how individual operations are composed together

> **Graph collections.** TensorFlow provides a general mechanism for storing collections of metadata in a `tf.Graph`, such as `tf.GraphKeys`, `tf.get_collection`

## Sessions
> A **`tf.Session`** object provides access to devices in the local machine, and remote devices using the distributed TensorFlow runtime. It also caches information about your tf.Graph so that you can efficiently run the same computation multiple times.

As session is also where the following questions are answered:
- on what server will the graph run?
- will the graph use a gpu
- cluster definition

**`tf.Session.run`** is the main execution function, 

### Using a Session

In [6]:
x = tf.constant([[1., 2.]])
y = tf.add(x, x)
print(y)

# create a session
with tf.Session() as sess:

    # evaluate the output and save to a variable
    result = sess.run(y)
print(result)

Tensor("Add:0", shape=(1, 2), dtype=float32)
[[2. 4.]]


### Interactive session

In [7]:
inter_sess = tf.InteractiveSession()

x = tf.constant([[1., 2.]])
y = tf.add(x, x)
print(y)
print(y.eval())
inter_sess.close()

Tensor("Add_1:0", shape=(1, 2), dtype=float32)
[[2. 4.]]


### Eager Execution

```python
import tensorflow as tf
tf.enable_eager_execution()
x = tf.constant([[1., 2.]])
y = tf.add(x, x)
print(y)

    tf.Tensor([[2. 4.]], shape=(1, 2), dtype=float32)
```

In [8]:
grf = tf.Graph()
print(grf)

## How to manage multiple graphs

>As noted above, TensorFlow provides a "default graph" that is implicitly passed to all API functions in the same context. **For many applications, a single graph is sufficient.**

> A `tf.Graph` defines the namespace for tf.Operation objects: each operation in a single graph **must have a unique name.**

In [9]:
graph_1 = tf.Graph()
with graph_1.as_default():
    # all these operations will go to graph 1
    c = tf.constant(5, name="Node-Graph1")
    d = c + c
    sess_1 = tf.Session()

graph_2 = tf.Graph()
with graph_2.as_default():
    # all these operations will go to graph 2
    e = tf.constant(6, name="Node-Graph2")

# can directly assign the graph to a
# session
sess_2 = tf.Session(graph=graph_2)

# Print all of the operations in the default graph.
g = tf.get_default_graph()
print(g.get_operations())

[<tf.Operation 'Const' type=Const>, <tf.Operation 'alpha' type=Const>, <tf.Operation 'beta' type=Const>, <tf.Operation 'gamma' type=Const>, <tf.Operation 'delta' type=Const>, <tf.Operation 'epsilon' type=Const>, <tf.Operation 'lambda' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Const_2' type=Const>, <tf.Operation 'Add_1' type=Add>]


## Types of Values in Tensorflow

- **Placeholder** - a value that is unassigned, but initialized by the session. Generally **inputs** and **outputs**
- **Variable** - a value that can change, typically weights or parameters of your machine learning model
- **Constant** - A value that doesn't change

In [10]:
m10 = tf.constant([[1.,3.], [5.,7.]], name='epsilon')
m11 = tf.constant([[10.,20.], [30.,40.]], name='epsilon')

### Operators

In [11]:
m_add = tf.add(m10, m11)
m_subtract = tf.subtract(m10, m11)
m_multiply = tf.multiply(m10, m11)
m_sq = tf.pow(m11, m10)
m_exp = tf.exp(m10)
m_sqrt = tf.sqrt(m10)
m_div = tf.div(m11, m10)
m_true_div = tf.truediv(m11, m10)
m_floordiv = tf.floordiv(m11,m10)
m_mod = tf.mod(m11,m10)

with tf.Session() as sess:
    result = sess.run([m_add, m_subtract, m_multiply, m_sq, m_exp, m_sqrt, m_div, m_true_div, m_floordiv, m_mod])

for item in result:
    print(item, type(item))

[[11. 23.]
 [35. 47.]] <class 'numpy.ndarray'>
[[ -9. -17.]
 [-25. -33.]] <class 'numpy.ndarray'>
[[ 10.  60.]
 [150. 280.]] <class 'numpy.ndarray'>
[[1.0000e+01 8.0000e+03]
 [2.4300e+07 1.6384e+11]] <class 'numpy.ndarray'>
[[   2.7182817   20.085537 ]
 [ 148.41316   1096.6332   ]] <class 'numpy.ndarray'>
[[1.        1.7320508]
 [2.236068  2.6457512]] <class 'numpy.ndarray'>
[[10.         6.6666665]
 [ 6.         5.714286 ]] <class 'numpy.ndarray'>
[[10.         6.6666665]
 [ 6.         5.714286 ]] <class 'numpy.ndarray'>
[[10.  6.]
 [ 6.  5.]] <class 'numpy.ndarray'>
[[0. 2.]
 [0. 5.]] <class 'numpy.ndarray'>


# Visualizing with Tensorboard

In [12]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
    print(sess.run(x))
writer.close() # close the writer when you’re done using it


5


In [13]:
# !python3 [yourprogram].py 
# tensorboard --logdir="./graphs" --port 6006
# go to : http://localhost:6006/